In [ ]:
# !/home/user/ollama/bin/ollama serve # running it via a seperate sbatch script

In [ ]:
# !/home/user/ollama/bin/ollama pull llama3.2:latest 

In [ ]:
# !/home/user/ollama/bin/ollama pull deepseek-r1:14b

In [ ]:
# !/home/user/ollama/bin/ollama run llama3.2:latest # running it via a seperate sbatch script or directly in node's terminal or via python script below

⠙ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠴ >>> Send a message (/? for help)

In [ ]:
!/home/user/ollama/bin/ollama ps

NAME               ID              SIZE     PROCESSOR    UNTIL              
deepseek-r1:14b    ea35dfe18182    11 GB    100% GPU     2 minutes from now    


In [ ]:
!/home/user/ollama/bin/ollama ls

NAME               ID              SIZE      MODIFIED       
deepseek-r1:14b    ea35dfe18182    9.0 GB    24 seconds ago    
llama3.2:latest    a80c4f17acd5    2.0 GB    2 months ago      


In [ ]:
# !pip install ollama

In [15]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='deepseek-r1:14b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

<think>

</think>

The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight reaches Earth's atmosphere, it interacts with molecules and small particles in the air. Sunlight is composed of various colors, each corresponding to different wavelengths. Blue light has a shorter wavelength compared to other colors like red or orange. Shorter wavelengths are scattered more by the atmosphere, causing blue light to be dispersed in all directions. As a result, we see the sky as blue because our eyes receive scattered blue light from all over the sky.
<think>

</think>

The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight reaches Earth's atmosphere, it interacts with molecules and small particles in the air. Sunlight is composed of various colors, each corresponding to different wavelengths. Blue light has a shorter wavelength compared to other colors like red or orange. Shorter wavelengths are scattered more by the atmosphere, causing blue 

In [ ]:
# !/home/user/ollama/bin/ollama stop llama3.2:latest

In [ ]:
!/home/user/ollama/bin/ollama stop deepseek-r1:14b

In [ ]:
%pip install "unstructured[all-docs]" chromadb pydantic lxml tiktoken langchain langchain-community langchain_ollama langchain-chroma

In [2]:
from unstructured.partition.pdf import partition_pdf
import pytesseract
from tqdm import tqdm
# from dotenv import load_dotenv
from langchain_ollama.llms import OllamaLLM #for standard
from langchain_ollama.chat_models import ChatOllama # for chat-based models
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import pickle

import os
import base64
import time

print("Imports Done!!")

Imports Done!!


In [23]:
pdf_path = "../data_ollama_deepseek_r1_14b/pdfs/nRF52840_PS_v1.11.pdf"
image_path = "../data_ollama_deepseek_r1_14b/images_collect/images_nrf52820"
db_path = "../data_ollama_deepseek_r1_14b/pdf_partitioning_result/chroma_dbs/chroma_db_nrf52820"
pickle_path = "../data_ollama_deepseek_r1_14b/pdf_partitioning_result/pickle_files/nrf52820.pkl"

In [ ]:
!pwd

In [5]:
model_name = "deepseek-r1:14b" 
embedd_model = "nomic-embedd-text"

In [8]:
def load_chroma_db(local_directory=db_path):
    embeddings = OllamaEmbeddings(model=embedd_model)
    return Chroma(persist_directory=local_directory, embedding_function=embeddings)

In [9]:
def categorize_elements(raw_pdf_elements):
    text_elements = []
    table_elements = []
    for element in raw_pdf_elements:
        if 'CompositeElement' in str(type(element)):
            text_elements.append(str(element))
        elif 'Table' in str(type(element)):
            table_elements.append(str(element))
    return text_elements, table_elements

In [12]:
model = ChatOllama(model=model_name,temperature=0, num_predict=1024) #kept parameters same as in case of openAI

In [13]:
# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well-optimized for retrieval. \
    Don't use Markdown, just plain text output. Table \
    or text: {element} """
    prompt = PromptTemplate.from_template(prompt_text)

    # Text summary chain
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries

In [14]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [15]:
def image_summarize(img_base64, prompt):
    """Make image summary"""
    msg = model.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

In [16]:
def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """
    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Include all the values in each image, including extracting all the text. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

In [17]:
def create_multi_vector_retriever(vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """
    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

In [ ]:
if os.path.exists(db_path) and os.path.exists(pickle_path):
    print("Loading existing Chroma database...")
    vectorstore = load_chroma_db()

    with open(pickle_path, 'rb') as f:
        loaded_data = pickle.load(f)

    # Access the variables
    texts = loaded_data['texts']
    tables = loaded_data['tables']
    text_summaries = loaded_data['text_summaries']
    table_summaries = loaded_data['table_summaries']
    img_base64_list = loaded_data['img_base64_list']
    image_summaries = loaded_data['image_summaries']

else:
    print("Creating new Chroma database...")
    # Store embeddings in Chroma
    start_time = time.time()
    pdf_elements = partition_pdf(
        pdf_path,
        chunking_strategy="by_title",
        extract_images_in_pdf=True,
        infer_table_structure=True,
        extract_image_block_types=['Table', 'Image'],
        extract_image_block_output_dir=image_path,
        max_characters=3000,
        new_after_n_chars=2800,
        combine_text_under_n_chars=2000,
        image_output_dir_path=image_path
    )
    end_time = time.time()
    print(f"pdf partition done!! Time : {end_time-start_time}")
    
    # extract tables and texts
    start_time = time.time()
    texts, tables = categorize_elements(pdf_elements)
    end_time = time.time()
    print(f"categorize elements done!! Time : {end_time - start_time}")
    
    # Get text & table summaries
    start_time = time.time()
    text_summaries, table_summaries = generate_text_summaries(texts[0:19], tables, summarize_texts=True)
    end_time = time.time()
    print(f"generate text summaries done!! Time : {end_time - start_time}")

    # Image summaries
    start_time = time.time()
    img_base64_list, image_summaries = generate_img_summaries(image_path)
    end_time = time.time()
    print(f"generate img summaries done!! Time : {end_time - start_time}")
    with open(pickle_path, 'wb') as f:
        pickle.dump({
            'texts': texts,
            'tables': tables,
            'text_summaries': text_summaries,
            'table_summaries': table_summaries,
            'img_base64_list': img_base64_list,
            'image_summaries': image_summaries
        }, f)
    print("Dumped pickle file")

    start_time = time.time()
    vectorstore = Chroma(
        collection_name="mm_rag",
        embedding_function =OllamaEmbeddings(model=embedd_model),
        persist_directory=db_path
    )
    end_time = time.time()
    print(f"vectorstore done!! Time : {end_time - start_time}")
print("Done!!")

Creating new Chroma database...


In [ ]:
# Stage 2 : After pdf partitioning & embedded vector creation

In [ ]:
# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

In [ ]:
import io
import re

from IPython.display import HTML, display
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from PIL import Image


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))

def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    if len(b64_images) > 0:
        return {"images": b64_images[:1], "texts": []}
    return {"images": b64_images, "texts": texts}

In [ ]:
## Prepare RAG Pipeline

In [ ]:
def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    messages = []

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are an AI scientist tasking with providing factual answers from a datasheet of a System-on-Chip (SoC) \n"
            "Use this information to provide answers related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
        ),
    }
    messages.append(text_message)
    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)
    return [HumanMessage(content=messages)]

def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model  # MM_LLM
        | StrOutputParser()
    )

    return chain

In [ ]:
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [ ]:
def ask_bot(query):
    return chain_multimodal_rag.invoke(query)

In [ ]:
response = ask_bot("What is the base address of UART")
print(response)